**BeautifulSoup**: là một thư viện của Python cho phép chúng ta lấy dữ liệu từ HTML đơn giản và hiệu quả

Tham khảo: https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3

# Import các thư viện
- requests: tìm nạp nội dung trang (gửi một yêu cầu HTTP GET đến url của trang web mà bạn cần, nó sẽ phản hồi lại với nội dung HTML) với 200 - gửi yêu cầu thành công
- BeautifulSoup: tìm nạp và phân tích dữ liệu và giữ nó trong các kiểu dữ liệu dict hoặc list thông thường

In [ ]:
import requests
from bs4 import BeautifulSoup

#Import Drive:
Kết nối google colab ta với My Drive của Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Tạo một thư mục ảo trên Google Colab

In [ ]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


#Tiến hành thu thập dữ liệu 
trang thu thập: 'https://www.thegioididong.com/dtdd'
thu thập được về 4679 bình luận cho tất cả sản phẩm điện thoại trên trang với các nhãn tự động (dự tính)

In [ ]:
import re
url='https://www.thegioididong.com/dtdd'
url1=url
#Mở file lên để ghi dữ liệu, nếu không có file thì file .json sẽ tự động được tạo
with open('/content/drive/My Drive/new_data_machinelearning.json','a')as file:
    # Gửi yêu cầu HTTP bằng request đến url1.
    r1=requests.get(url1)
    # phân tích nội dung html 
    coverpage=r1.content
    # tạo một đối tượng phân tích cú pháp - soup, với sự trợ giúp của trình phân tích "html.parser" được tích hợp sẵn của BeautifulSoup và Python
    soup = BeautifulSoup(coverpage, 'html.parser')
    # ta tiến hành tìm đến các <div class="manu manu14"> chứa html của các hãng điện thoại (iphone, oppo, samsung, xiaomi, realme,...) 
    coverpage_news=soup.find('div',class_='manu manu14') 
    # tiếp tục tìm đến các mã nguồn 'a' để lấy link của các hãng điện thoại trong nó
    test5=coverpage_news.find_all('a')
    # mình phải thực hiện remove test5[8], vì trong quá trình lấy, vô tình lấy luôn cái mã nguồn button(xem thêm) trên trang
    test5.remove(test5[8])
    a1=[]
    nho=''
    url='https://www.thegioididong.com'
    for i in range (len(test5)):
      b1=url+test5[i]['href'] # lấy link-'href' trên mã nguồn
      a1.append(b1)
    print(a1)
    print(len(a1))
    # ứng với mỗi đường link hãng điện thoại, ta chạy vòng lặp thu thập các đường link sản phẩm điện thoại 
    for k in range(len(a1)):
      ni=[]
      lol=[]
      r2=requests.get(a1[k])
      coverpage=r2.content
      soup = BeautifulSoup(coverpage, 'html.parser')
      #ta tiến hãnh tìm kiếm đến các <ul class="homeproduct filter-cate"> chứa các đường link của các sản phẩm điện thoại trên trang
      coverpage_news=soup.find('ul',class_='homeproduct filter-cate ')
      # tiếp tục tìm kiếm đến các mã nguồn 'a' chứa trực tiếp đường link trên trang
      test=coverpage_news.find_all('a')
      # mình thực hiện tìm đến 'h3' để get_text lấy tên của các sản phẩm điện thoại trên trang
      for te in test:
        te1=te.find_all('h3')
        lol.append(te1[0].get_text())
      # thực hiện lấy link các sản phẩm điện thoại trên trang
      for m in range (len(test)):
        b2=url+test[m]['href']
        ni.append(b2)
      # ứng với mỗi đường link sản phẩm điện thoại, ta chạy vòng lặp thu thập các bình luận của nó
      for l in range (len(ni)):
        # thực hiện + mỗi đường link ở trên với '/danh-gia' để chuyển đến trang "Xem tất cả đánh giá" sản phẩm ứng với mỗi điện thoại
         r3=requests.get(ni[l]+'/danh-gia')
         print(ni[l])
         coverpage=r3.content
         soup = BeautifulSoup(coverpage, 'html.parser')
         a=[] # 1 list lưu các bình luận ứng với mỗi sản phẩm điện thoại
         d=[] # 1 list có chứa các mã nguồn số sao ứng với mỗi bình luận
         number=[] # 1 list lưu số lượng được 5 sao ứng với mỗi sản phẩm điện thoại 
         # ta thực hiện tìm đến <ul class="ratingLst">, chứa toàn bộ các bình luận trên trang
         coverpage_news = soup.find_all('ul', class_= "ratingLst")
         for test in coverpage_news:
           # ứng với mỗi sản phẩm có một id - tìm đến <li class="par"> 
           test1=test.find_all('li',class_="par")
           for test_ in test1:
             # tiếp tục tìm đến <div class="rc"> chứa trực tiếp các bình luận của sản phẩm
             test2=test_.find_all('div',class_='rc')
             # thực hiện lấy bình luận (get_text()) ứng với mỗi sản phẩm và đưa vào list a
             b=test2[0].get_text()
             # tìm đến các 'span' chứa các mã nguồn số sao lưu vào list d
             for test3 in test2:
               test4=test3.find_all('span')
               d.append(test4[0])
             b = re.sub('[!@#$\n""'').\*]', '', b) #loại bỏ các kí tự không cần thiết
             a.append(b)
        # đếm các '<i class="iconcom-txtstar"></i>' để ra được số lượng đạt 5 sao của bình luận và đưa vào list number
         for w in range (len(d)):
           number.append(str(d[w]).count('<i class="iconcom-txtstar"></i>'))
         for t in range(len(a)):
           print(a[t]) # In ra bình luận của sản phẩm
           print("SỐ LƯỢNG 5 SAO : ",number[t]) # In ra số lượng đạt 5 sao
           print(lol[l]) # In ra tên của sản phẩm
           # Bắt đầu thực hiện ghi file
           if a[t]!=nho:
            file.write('{"TYPE": "')
            # thực hiện gán nhãn tự động:
            # với các bình luận có số lượng đạt 5 sao là 5 ta gán nhãn là 1
            # với các bình luận có số lượng đạt 5 sao là 4 hoặc 3 ta gán nhãn là 0
            # với các bình luận có số lượng đạt 5 sao là 1 hoặc 2 ta gán nhãn là -1
            if number[t]==5:
              file.write("1")
            elif number[t]==4 or number[t]==3:
              file.write("0")
            else:
              file.write("-1")
            file.write('","NAME": "')
            file.write(lol[l])
            file.write('","COMMENT": "')
            file.write(a[t])
            file.write('", "RATING COMMENT": "')
            file.write(str(number[t]))
            file.write('"}\n')
           nho=a[t]
    file.close()

Streaming output truncated to the last 5000 lines.
SỐ LƯỢNG 5 SAO :  5
Realme 5s
Mới mua tuyệt vời nhân viên dmx từ vẫn nhiệt tình và chứ đáo còn về máy sẽ trải nghiệm thử
SỐ LƯỢNG 5 SAO :  5
Realme 5s
Mình mua được 2 ngày nói chung điện thoại ổn pin để qua đêm tuột 4% không biết như vậy là bình thường hay là có sao hay khong nữa 
SỐ LƯỢNG 5 SAO :  4
Realme 5s
pin trâu bò  chiến pubg thì ổn lâu lâu giật tý nói chung tầm giá này khá ổn🥰
SỐ LƯỢNG 5 SAO :  5
Realme 5s
Sản phẩm tốt, máy rất mượt và tầm 3 ngày mình mới sạc 1 lần Và sạc cũng nhanh, Camera chụp thì tương đối nét  Nhân viên TGDĐ nhiệt tình
SỐ LƯỢNG 5 SAO :  5
Realme 5s
Sản phẩm thật sự tốt Nếu ai như mình xài đt chỉ để lướt web, mạng xã hội, chụp hình, chỉnh ảnh, youtube các kiểu thì máy rất mượt và tầm 3 ngày mình mới sạc 1 lần Và sạc cũng nhanh chứ không quá lâu cho viên pin 5000 như thế này Camera chụp nét và độ tương phản tốt Bên cạnh đó cũng cảm ơn nhân viên TGDĐ nhiều vì sự nhiệt tình 
SỐ LƯỢNG 5 SAO :  5
Realme 5s
Mới m